# Snap through example
Derivation of the model based on a modified Von Mises Arch from Fig2 of:
https://doi.org/10.1016/j.ijsolstr.2020.11.037

In [12]:
using Pkg
Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `c:\Users\tim\Documents\2025FallExamples\10-22`


In [13]:
k1, k2, lo, wo, ho = symbols("k_1 k_2 l_o w_o h_o", real=true, positive=true)
θ, ua, up, fa = symbols("theta u_a u_p f_a", real=true)
κ, α = symbols("kappa alpha", real=true, positive=true)
nondim = Dict(k2 => κ * k1, lo => α * ho, wo => ho * sqrt(α^2 - 1))
L = sqrt(wo^2 + (ho - up)^2)
vals = Dict(κ => 1 / 2, α => 5 / 4)

Dict{Sym{PyCall.PyObject}, Float64} with 2 entries:
  alpha => 1.25
  kappa => 0.5

In [14]:
V = 2*1//2*k1*(L-lo)^2 + 1//2*k2*(up-ua)^2

                                     2                    
    /          _____________________\                    2
    |         /    2              2 |    k_2*(-u_a + u_p) 
k_1*\-l_o + \/  w_o  + (h_o - u_p)  /  + -----------------
                                                 2        

In [15]:
eq1 = diff(V,up) |> simplify

                  /         _____________________\                      ______ >
                  |        /    2              2 |                     /    2  >
2*k_1*(h_o - u_p)*\l_o - \/  w_o  + (h_o - u_p)  / - k_2*(u_a - u_p)*\/  w_o   >
------------------------------------------------------------------------------ >
                                     _____________________                     >
                                    /    2              2                      >
                                  \/  w_o  + (h_o - u_p)                       >

> _______________
>              2 
> + (h_o - u_p)  
> ---------------
>                
>                
>                

In [16]:
eq2 = diff(V,ua)-fa |> simplify

-f_a + k_2*(u_a - u_p)

In [17]:
eq1bar = eq1/ho/k1 |> subs(nondim) |> subs(up => up * ho, ua => ua * ho, fa => fa * k1 * ho) |> simplify |> expand |> simplify

                            _______________________                            >
                           /      2      2                                     >
-2*alpha*u_p + 2*alpha + \/  alpha  + u_p  - 2*u_p *(-kappa*u_a + kappa*u_p +  >
------------------------------------------------------------------------------ >
                                  _______________________                      >
                                 /      2      2                               >
                               \/  alpha  + u_p  - 2*u_p                       >

>           
>           
> 2*u_p - 2)
> ----------
>           
>           
>           

In [18]:
eq2bar = eq2 / ho / k1 |> subs(nondim) |> subs(up => up * ho, ua => ua * ho, fa => fa * k1 * ho) |> simplify

-f_a + kappa*(u_a - u_p)

In [19]:
R = [eq1bar, eq2bar]

2-element Vector{Sym{PyCall.PyObject}}:
 (-2*alpha*u_p + 2*alpha + sqrt(alpha^2 + u_p^2 - 2*u_p)*(-kappa*u_a + kappa*u_p + 2*u_p - 2))/sqrt(alpha^2 + u_p^2 - 2*u_p)
                                                                                                    -f_a + kappa*(u_a - u_p)

In [20]:
Fu = [ diff(r,u) |> simplify for r in R, u in [ua,up] ]

2×2 Matrix{Sym{PyCall.PyObject}}:
 -kappa  …  ((u_p - 1)*(alpha^2 + u_p^2 - 2*u_p)*(2*alpha*u_p - 2*alpha + sqrt(alpha^2 + u_p^2 - 2*u_p)*(kappa*u_a - kappa*u_p - 2*u_p + 2)) + ((-2*alpha + (kappa + 2)*sqrt(alpha^2 + u_p^2 - 2*u_p))*sqrt(alpha^2 + u_p^2 - 2*u_p) - (u_p - 1)*(kappa*u_a - kappa*u_p - 2*u_p + 2))*(alpha^2 + u_p^2 - 2*u_p)^(3/2))/(alpha^2 + u_p^2 - 2*u_p)^(5/2)
  kappa                                                                                                                                                                                                                                                                                                                                         -kappa

In [21]:
Fλ = [diff(r,λ) |> simplify for r in R, λ in [fa] ]

2×1 Matrix{Sym{PyCall.PyObject}}:
  0
 -1